In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 12854713121253401334
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 5763592192
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17031650478744042123
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:0b:00.0, compute capability: 7.5"
xla_global_id: 416903419
]


I0000 00:00:1737574193.471733    3414 gpu_device.cc:2022] Created device /device:GPU:0 with 5496 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 2080 SUPER, pci bus id: 0000:0b:00.0, compute capability: 7.5


In [1]:
import pandas as pd

df = pd.read_parquet('../data/train_preprocessed_small.parquet')
display(df)

,label,preprocessed_text
0,1,house dem aide comey letter jason chaffetz twe...
1,0,flynn hillary clinton big woman campus breitba...
2,1,truth fire truth fire october tension intellig...
3,1,civilian kill single airstrike identify video ...
4,1,iranian woman jail fictional unpublished story...
...,...,...
20795,0,rapper t trump poster child white supremacy ra...
20796,0,n f l playoff schedule matchup odd new york ti...
20797,0,macys receive takeover approach hudsons bay ne...
20798,1,nato russia hold parallel exercise balkans nat...


In [2]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(df["preprocessed_text"], df['label'], test_size=0.3, random_state=42)

X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.3, random_state=42)

print(f"Train size: {len(X_train)}")
print(f"Validation size: {len(X_val)}")
print(f"Test size: {len(X_test)}")

Train size: 14142
Validation size: 4242
Test size: 1819


In [3]:
from transformers import RobertaTokenizer, TFRobertaForSequenceClassification, AdamW
import tensorflow as tf

roberta_tokenizer = RobertaTokenizer.from_pretrained("roberta-base", do_lower_case=True)
roberta_model = TFRobertaForSequenceClassification.from_pretrained("roberta-base", #RoBERTa base model
                                                                    num_labels = 2,  #number of output labels - 0,1 (binary classification)
                                                                    output_attentions = False,  #model doesnt return attention weights
                                                                    output_hidden_states = False #model doesnt return hidden states
                                                                )


# roberta_model.cuda()
print(' Base models loaded') 

/home/nikl/programming/fhdw/knowledge_engineering_ausarbeitung/fakenews_detection/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-21 14:28:24.771656: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-21 14:28:24.863910: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-21 14:28:24.951399: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737466105.023110   11613 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been register

 Base models loaded


In [4]:
display(X_train)

16677    james stavridis retire admiral vet hillary cli...
14013    state swat team drill prep backlash steal elec...
14575    george michaels death meet disbelief celebrity...
5629     build cute bee hotel help population event chr...
8275     new comment feature add comment leave reply cl...
                               ...                        
11625    chaiwali indian restaurant feel like home new ...
12329    recipe mouth watering cauliflower coconut oil ...
5553     las cajetillas de tabaco emitir n m sica de me...
885      open border group gird h b fight open border l...
16251    eighty wealthy new yorker ask state government...
Name: preprocessed_text, Length: 14142, dtype: object

In [5]:
# Split the sentence into tokens -RoBERTa
print('Tokenized RoBERT: ', roberta_tokenizer.tokenize(X_train.iloc[0]))

Tokenized RoBERT:  ['j', 'ames', 'Ġst', 'av', 'rid', 'is', 'Ġretire', 'Ġadm', 'iral', 'Ġvet', 'Ġhill', 'ary', 'Ġcl', 'inton', 'Ġrun', 'Ġmate', 'Ġnew', 'Ġy', 'ork', 'Ġtimes', 'Ġhill', 'ary', 'Ġcl', 'inton', 'Ġcampaign', 'Ġvet', 'Ġj', 'ame', 'Ġg', 'Ġst', 'av', 'rid', 'is', 'Ġretire', 'Ġnavy', 'Ġadm', 'iral', 'Ġserve', 'Ġth', 'Ġsupreme', 'Ġally', 'Ġcommander', 'Ġn', 'ato', 'Ġpossible', 'Ġrunning', 'Ġmate', 'Ġaccord', 'Ġperson', 'Ġknowledge', 'Ġvetting', 'Ġprocess', 'Ġclose', 'Ġm', 'rs', 'Ġcl', 'inton', 'Ġsecretary', 'Ġstate', 'Ġlikely', 'Ġmilitary', 'Ġexperience', 'Ġshort', 'list', 'Ġm', 'r', 'Ġst', 'av', 'rid', 'is', 'Ġcurrently', 'Ġdean', 'Ġf', 'letcher', 'Ġschool', 'Ġtu', 'fts', 'Ġuniversity', 'Ġfit', 'Ġdescription', 'Ġyear', 'Ġnat', 'os', 'Ġsupreme', 'Ġally', 'Ġcommander', 'Ġoversee', 'Ġoperation', 'Ġmiddle', 'Ġeast', 'Ġaf', 'ghan', 'istan', 'Ġlib', 'ya', 'Ġsy', 'ria', 'Ġbalk', 'an', 'Ġpiracy', 'Ġcoast', 'Ġaf', 'rica', 'Ġcl', 'inton', 'Ġcampaign', 'Ġdecline', 'Ġrequest', 'Ġcomment', '

In [6]:
# Mapping tokens to token IDs - RoBERTa
print('Token IDs RoBERTa: ', roberta_tokenizer.convert_tokens_to_ids(roberta_tokenizer.tokenize(X_train.iloc[0])))

Token IDs RoBERTa:  [267, 12336, 1690, 1469, 10505, 354, 7865, 20087, 24811, 9087, 9910, 1766, 3741, 10528, 422, 12563, 92, 1423, 9657, 498, 9910, 1766, 3741, 10528, 637, 9087, 1236, 4344, 821, 1690, 1469, 10505, 354, 7865, 13504, 20087, 24811, 1807, 3553, 15835, 7564, 8501, 295, 3938, 678, 878, 12563, 10170, 621, 2655, 23371, 609, 593, 475, 4926, 3741, 10528, 2971, 194, 533, 831, 676, 765, 8458, 475, 338, 1690, 1469, 10505, 354, 855, 19955, 856, 42795, 334, 13145, 21811, 2737, 2564, 8194, 76, 23577, 366, 15835, 7564, 8501, 13752, 2513, 1692, 3017, 9724, 5158, 7566, 21748, 2636, 13550, 6374, 31110, 260, 33551, 3673, 9724, 14962, 3741, 10528, 637, 2991, 2069, 1129, 475, 338, 1690, 1469, 10505, 354, 2991, 1129, 9115, 637, 621, 2655, 23371, 1994, 20797, 5685, 2574, 609, 475, 338, 1690, 1469, 10505, 354, 4830, 21559, 831, 3054, 3598, 1141, 5451, 537, 18521, 29868, 6664, 2389, 339, 18362, 423, 699, 6046, 251, 8027, 11408, 803, 1504, 5623, 680, 1805, 1141, 1354, 985, 8027, 11408, 14253, 937,

In [7]:
train_encoded_roberta = roberta_tokenizer(X_train.to_list(), padding=True, truncation=True, max_length=512, return_tensors="tf")
val_encoded_roberta = roberta_tokenizer(X_val.to_list(), padding=True, truncation=True, max_length=512, return_tensors="tf")
test_encoded_roberta = roberta_tokenizer(X_test.to_list(), padding=True, truncation=True, max_length=512, return_tensors="tf")


In [8]:
y_train = tf.convert_to_tensor(y_train.to_list(), dtype=tf.int32)
y_val = tf.convert_to_tensor(y_val.to_list(), dtype=tf.int32)
y_test = tf.convert_to_tensor(y_test.to_list(), dtype=tf.int32)


In [9]:
train_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": train_encoded_roberta["input_ids"], 
     "attention_mask": train_encoded_roberta["attention_mask"]}, 
    y_train
)).batch(32)

val_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": val_encoded_roberta["input_ids"], 
     "attention_mask": val_encoded_roberta["attention_mask"]}, 
    y_val
)).batch(32)

test_dataset = tf.data.Dataset.from_tensor_slices((
    {"input_ids": test_encoded_roberta["input_ids"], 
     "attention_mask": test_encoded_roberta["attention_mask"]}, 
    y_test
)).batch(32)


In [1]:
import tensorflow as tf
print("Num GPUs Available:", len(tf.config.list_physical_devices('GPU')))


2025-01-21 14:53:04.130540: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-21 14:53:04.133357: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-01-21 14:53:04.142423: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1737467584.157127    1095 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1737467584.161536    1095 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-21 14:53:04.176183: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU ins

Num GPUs Available: 0


W0000 00:00:1737467586.433070    1095 gpu_device.cc:2344] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [ ]:
train_dataset = train_dataset.batch(8)  # Reduce batch size
val_dataset = val_dataset.batch(8)


In [ ]:
from tensorflow.keras.mixed_precision import set_global_policy
set_global_policy("mixed_float16")


In [10]:
# Compile the model
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metrics = ["accuracy"]

roberta_model.compile(optimizer=optimizer, loss=loss, metrics=metrics)

# Train the model
roberta_model.fit(train_dataset, validation_data=val_dataset, epochs=3)


Epoch 1/3


: 